<a href="https://colab.research.google.com/github/DarkTHor55/learning-ml/blob/main/practice%202.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd



In [2]:
df=pd.read_csv("/content/used_cars.csv")

In [3]:
df.sample(20)

,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
2948,Volvo,XC90 T6 Inscription,2019,"60,000 mi.",Gasoline,316.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,8-Speed A/T,White,Black,None reported,Yes,"$37,000"
889,Chevrolet,Camaro 2LT,2015,"56,675 mi.",Gasoline,323.0HP 3.6L V6 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Silver,Black,None reported,Yes,"$19,750"
2606,Chevrolet,Express 2500 Work Van,2017,"76,000 mi.",Gasoline,285.0HP 4.8L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,–,None reported,Yes,"$41,400"
3371,Porsche,Cayenne Turbo GT,2022,"1,766 mi.",Gasoline,4.0L,Automatic,Custom Color,Black,None reported,NaN,"$209,995"
3842,RAM,1500 Laramie,2022,"15,741 mi.",Gasoline,5.7L V8 16V MPFI OHV,8-Speed Automatic,Flame Red Clearcoat,Black,None reported,NaN,"$45,999"
946,Lincoln,Navigator Reserve,2022,"13,750 mi.",Gasoline,440.0HP 3.5L V6 Cylinder Engine Gasoline Fuel,10-Speed A/T,White,Beige,None reported,Yes,"$79,999"
3872,Ford,F-150 XLT,2016,"71,000 mi.",E85 Flex Fuel,385.0HP 5.0L 8 Cylinder Engine Flex Fuel Capab...,A/T,Black,–,At least 1 accident or damage reported,Yes,"$24,350"
336,BMW,540 i xDrive,2020,"37,426 mi.",Gasoline,2.0 Liter Turbo,Automatic,White,Black,None reported,NaN,"$39,998"
1053,Nissan,Armada Platinum,2018,"108,206 mi.",Gasoline,5.6 Liter,Automatic,White,Charcoal,At least 1 accident or damage reported,NaN,"$26,498"
3212,BMW,325 i,2003,"142,000 mi.",Gasoline,184.0HP 2.5L Straight 6 Cylinder Engine Gasoli...,5-Speed M/T,Silver,Black,At least 1 accident or damage reported,Yes,"$4,800"


In [4]:
df.shape

(4009, 12)

In [5]:
df.isnull().sum()

,0
brand,0
model,0
model_year,0
milage,0
fuel_type,170
engine,0
transmission,0
ext_col,0
int_col,0
accident,113


In [6]:
df['clean_title'] = df['clean_title'].fillna('No')

In [7]:
# Case 1: clean_title == 'No'
df.loc[
    df['accident'].isna() & (df['clean_title'] == 'No'),
    'accident'
] = 'None reported'

# Case 2: clean_title != 'No'
df.loc[
    df['accident'].isna() & (df['clean_title'] != 'No'),
    'accident'
] = 'At least 1 accident or damage reported'


In [8]:
df['fuel_type'] = df['fuel_type'].fillna('Unknown')
df['fuel_type'] = df['fuel_type'].replace(['–', 'not supported'], 'Unknown')

In [9]:
df['milage'] = (
    df['milage']
    .str.replace(r'[^0-9]', '', regex=True)
    .astype(int)
)

In [10]:
df['price'] = (
    df['price']
    .str.replace(r'[^0-9]', '', regex=True)
    .astype(int)
)

In [11]:
def merge_rare(df, col, threshold):
    freq = df[col].value_counts()
    df[col] = df[col].replace(freq[freq <= threshold].index, 'Others')

merge_rare(df, 'int_col', 150)
merge_rare(df, 'ext_col', 200)
merge_rare(df, 'transmission', 200)
merge_rare(df, 'engine', 40)
merge_rare(df, 'model', 15)
merge_rare(df, 'brand', 100)


In [12]:
from sklearn.model_selection import train_test_split

X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [13]:
cat_cols = [
    'clean_title','accident','fuel_type','brand',
    'model','engine','transmission','ext_col','int_col'
]

num_cols = ['model_year', 'milage']


In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), cat_cols),
        ('num', StandardScaler(), num_cols)
    ]
)


In [15]:
X_train_final = preprocessor.fit_transform(X_train)
X_test_final  = preprocessor.transform(X_test)


In [16]:
from sklearn.linear_model import LinearRegression

model = LinearRegression()
model.fit(X_train_final, y_train)


LinearRegression()

In [17]:
from sklearn.metrics import mean_absolute_error, r2_score

y_pred = model.predict(X_test_final)

mae = mean_absolute_error(y_test, y_pred)
r2  = r2_score(y_test, y_pred)

print("MAE:", mae)
print("R2 :", r2)


MAE: 30075.51160390762
R2 : 0.038165391078998634
